<a href="https://colab.research.google.com/github/ykitaguchi77/test/blob/main/Montecarlo-simulator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#**モンテカルロ法シミュレーター**

このPythonスクリプトは、バカラゲームにおけるモンテカルロ法を用いたベッティング戦略のシミュレーションを実行します。

## 目的

このスクリプトは、友人に誘われてカジノでバカラをする際に、どれぐらいの予算を持っていけば良いかをシミュレーションするためのものです。モンテカルロ法は利益を出しやすい方法とされていますが、試行回数が増えると大嵌りして財産を失うリスクも増大します。やめるタイミングを見極める目安にご活用ください！

## 概要

モンテカルロ法は、特定の数列に基づいてベット額を決定し、勝敗に応じて数列を変更しながら利益の最大化を目指す戦略です。このシミュレーションでは、バカラのルールを簡略化し、勝率50%の条件下でこの戦略の効果を検証します。

## 主な機能

1. **初期数列の生成**: 初期投資額に基づいて、3つの要素からなる初期数列を生成します。
2. **ベッティングロジック**: 数列の最初と最後の要素の和をベット額とします。
3. **数列の更新**: 勝利時は両端の要素を削除し、敗北時は賭け金を数列の末尾に追加します。
4. **数列のリセット**: 数列が1つまたは0つの要素になった場合、初期数列に戻します。
5. **複数回のシミュレーション**: 指定された回数のシミュレーションを実行し、統計情報を収集します。

## 実装の詳細

### 主要な関数

1. `generate_initial_sequence(initial_investment)`:
   - 初期投資額から初期数列を生成します。
   - 例: 初期投資額が4000の場合、[1000, 2000, 3000]を返します。

2. `baccarat_monte_carlo(initial_investment, initial_balance, max_rounds)`:
   - 1回のシミュレーションを実行します。
   - 各ラウンドでベットを行い、勝敗に応じて残高と数列を更新します。
   - 数列のリセット回数も追跡します。

3. `run_simulation(initial_investment, initial_balance, max_rounds, num_simulations)`:
   - 指定された回数のシミュレーションを実行し、結果を集計します。

### 主要なパラメータ

- `initial_investment`: 初期投資額（初期数列の生成に使用）
- `initial_balance`: 初期残高
- `max_rounds`: 1回のシミュレーションの最大ラウンド数
- `num_simulations`: シミュレーションの実行回数

## 使用方法

1. スクリプトを実行します。
2. プロンプトに従って、初期投資額、初期残高、最大ラウンド数、シミュレーション回数を入力します。
3. シミュレーション結果が表示されます。

## 出力情報

- 初期の数列
- 平均最終残高
- 破産確率（ゼロになる確率）
- 元金を下回る確率
- 利益を出せる確率
- 利益が出た場合の平均最終額
- 利益が出た場合の最高額
- 平均数列リセット回数
- 入力したパラメータ（初期投資額、初期残高、最大ラウンド数、シミュレーション回数）

In [44]:
import random

def generate_initial_sequence(initial_investment):
    base = initial_investment // 4
    return [base, base * 2, base * 3]

def baccarat_monte_carlo(initial_investment, initial_balance, max_rounds):
    initial_sequence = generate_initial_sequence(initial_investment)
    sequence = initial_sequence.copy()
    balance = initial_balance

    for _ in range(max_rounds):
        if balance <= 0:
            break

        if not sequence:
            sequence = initial_sequence.copy()

        bet = sequence[0] + sequence[-1]
        if bet > balance:
            bet = balance

        balance -= bet

        if random.random() < 0.5:  # Win (50% chance)
            balance += bet * 2
            if len(sequence) > 1:
                sequence = sequence[1:-1]
            else:
                sequence = []
        else:  # Lose
            sequence.append(bet)

    return balance

def run_simulation(initial_investment, initial_balance, max_rounds, num_simulations):
    final_balances = []
    bankruptcies = 0
    below_initial = 0
    profitable_balances = []

    for _ in range(num_simulations):
        final_balance = baccarat_monte_carlo(initial_investment, initial_balance, max_rounds)
        final_balances.append(final_balance)

        if final_balance == 0:
            bankruptcies += 1
        if final_balance < initial_balance:
            below_initial += 1
        if final_balance > initial_balance:
            profitable_balances.append(final_balance)

    avg_final_balance = sum(final_balances) / num_simulations
    bankruptcy_rate = bankruptcies / num_simulations * 100
    below_initial_rate = below_initial / num_simulations * 100
    profit_rate = (len(profitable_balances) / num_simulations) * 100

    avg_profitable_balance = sum(profitable_balances) / len(profitable_balances) if profitable_balances else 0
    max_profitable_balance = max(profitable_balances) if profitable_balances else 0
    min_final_balance = min(final_balances)

    return avg_final_balance, bankruptcy_rate, below_initial_rate, profit_rate, avg_profitable_balance, max_profitable_balance, min_final_balance

# ユーザー入力
initial_investment = 4000
initial_balance = 60000
max_rounds = 50
num_simulations = 10000

# 初期数列の生成
initial_sequence = generate_initial_sequence(initial_investment)

# シミュレーション実行
avg_final_balance, bankruptcy_rate, below_initial_rate, profit_rate, avg_profitable_balance, max_profitable_balance, min_final_balance = run_simulation(initial_investment, initial_balance, max_rounds, num_simulations)

# 結果表示
print(f"\nバカラ・モンテカルロ法シミュレーション結果:")
print(f"初期の数列: {initial_sequence}")
print(f"平均最終残高: {avg_final_balance:.2f}")
print(f"破産確率 (ゼロになる確率): {bankruptcy_rate:.2f}%")
print(f"元金を下回る確率: {below_initial_rate:.2f}%")
print(f"利益を出せる確率: {profit_rate:.2f}%")
print(f"利益が出た場合の平均最終額: {avg_profitable_balance:.2f}")
print(f"利益が出た場合の最高額: {max_profitable_balance:.2f}")
print(f"最終の最低残高: {min_final_balance:.2f}")
print(f"初期投資額: {initial_investment}")
print(f"初期残高: {initial_balance}")
print(f"最大ラウンド数: {max_rounds}")
print(f"シミュレーション回数: {num_simulations}")


バカラ・モンテカルロ法シミュレーション結果:
初期の数列: [1000, 2000, 3000]
平均最終残高: 59265.60
破産確率 (ゼロになる確率): 47.61%
元金を下回る確率: 51.37%
利益を出せる確率: 48.40%
利益が出た場合の平均最終額: 119339.67
利益が出た場合の最高額: 192000.00
最終の最低残高: 0.00
初期投資額: 4000
初期残高: 60000
最大ラウンド数: 50
シミュレーション回数: 10000
